In [ ]:
# import block
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from scipy.spatial import distance

from hw2 import *

In [ ]:
# import data
records = pd.read_csv("Ecommerce.csv")
records.head(10)